In [ ]:
# Prepare data and imports

In [ ]:
# Calculate direct distances between starting positions for each player to the triggered wall

In [ ]:
# Define main function

In [4]:
import json
import numpy as np
import pandas as pd
import parse_data.prepare_data as prepare_data
import globals
import scipy
import math
import data_extraction.get_indices as get_indices
from assign_coordinates_to_walls import alcove_x, alcove_y
from parse_data import preprocess


# Prepare data - load json data and convert it into df and list of trials
data_folder = '/Users/benny/Desktop/MSc/Project/Git/repos/octagon_analysis/Json data'
json_filenames = ['2024-09-13_11-31-00_YansuJerrySocial.json']

df, trial_list = prepare_data.prepare_data(data_folder, json_filenames)


def direct_distance(trial_index, chosen_player):
  
    this_trial = trial_list[trial_index]

    # isolate slice onset event, trigger event, and activating client
    slice_onset_event = this_trial[this_trial['eventDescription'] == globals.SLICE_ONSET]
    trigger_event = this_trial[this_trial['eventDescription'] == globals.SELECTED_TRIGGER_ACTIVATION]
    
    # find index of slice onset and trigger event normalised to this trial
    slice_onset_idx = slice_onset_event.index[0]
    slice_onset_idx = int(slice_onset_idx - this_trial.index[0])
    trigger_idx = trigger_event.index[0]
    trigger_idx = int(trigger_idx - this_trial.index[0])

    # identify wall touched by winning player at trigger event 
    wall_triggered = trigger_event[globals.WALL_TRIGGERED].item()

    # define wall centers by finding midpoint between left and right edge of alcove 
    alcove_centers = {}
    num_walls = 8
    for i in range(num_walls):
        # Get the x and y coordinates for the current wall
        x1, x2 = alcove_x[i * 2], alcove_x[i * 2 + 1]
        y1, y2 = alcove_y[i * 2], alcove_y[i * 2 + 1]
        
        # Calculate the center point
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        
        # Store the center coordinates
        alcove_centers[i + 1] = (center_x, center_y)
   
    # Find wall center for the triggered wall
    alcove_center = alcove_centers[wall_triggered]
    
    num_players = preprocess.num_players(this_trial)
  
    # assign labels to values from this_trial from slice onset onwards
    coordinate_array_labels = []
    for i in range(num_players):
        coordinate_array_labels.extend((globals.PLAYER_LOC_DICT[i]['xloc'], globals.PLAYER_LOC_DICT[i]['yloc'])) 
    coordinate_arrays = {label : this_trial[label].values[slice_onset_idx:] for label in coordinate_array_labels}

    # extract x and y values at the start looping over players
    distances = []
    for i in range(num_players):
        if chosen_player is None:
           x = coordinate_arrays[globals.PLAYER_LOC_DICT[i]['xloc']][0]  # Player's x position
           y = coordinate_arrays[globals.PLAYER_LOC_DICT[i]['yloc']][0]  # Player's y position
           starting_position = np.array([x, y])
    
           # calculate distances using alcove_center and starting position arrats
           distance = [np.linalg.norm(np.array(alcove_center) - starting_position)]
           distances.append(distance)
            
           print(f"Trial {trial_index}, Player {i}: x = {x}, y = {y}, distance = {distance}")
        
        else:
           if i != chosen_player:
              pass
           else:
              x = coordinate_arrays[globals.PLAYER_LOC_DICT[i]['xloc']][0]  # Player's x position
              y = coordinate_arrays[globals.PLAYER_LOC_DICT[i]['yloc']][0]  # Player's y position
              starting_position = np.array([x, y])
    
              # calculate distances using alcove_center and starting position arrats
              distance = [np.linalg.norm(np.array(alcove_center) - starting_position)]
              distances.append(distance)
               
              print(f"Trial {trial_index}, Player {i}: x = {x}, y = {y}, distance = {distance}")

    return distances
# above as stand-alone code not functional, needs below to execute but I'm not actually sure why, because it needs to loop over trials I guess?

# TEST----------
# loop over trials within trial_list and return distances for chosen player using direct_distance function, prints Error if not 
for trial_index in range(len(trial_list)): 
    try:
        distance = direct_distance(trial_index, chosen_player = 0)
        print("Distances from starting positions to the alcove center:", distance)
    except Exception as e:
        print(f"Error: {e}")

Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
Trial 0, Player 0: x = -1.6792796850204468, y = 3.300809621810913, distance = [np.float64(21.65911490762354)]
Distances from starting positions to the alcove center: [[np.float64(21.65911490762354)]]
Trial 1, Player 0: x = 5.654138088226318, y = 15.102668762207031, distance = [np.float64(20.64295310499839)]
Distances from starting positions to the alcove center: [[np.float64(20.64295310499839)]]
Trial 2, Player 0: x = 13.669239044189453, y = 4.289867877960205, distance = [np.float64(33.097156075426355)]
Distances from starting positions to the alcove center: [[np.float64(33.097156075426355)]]
Trial 3, Player 0: x = -5.356249809265137, y = -16.14360809326172, distance = [np.float64(21.613188430252805)]
Distances from starting positions to the alcove center: [[np.float64(21.613188430252805)]]
Trial 4, Player 0: x = -14.024356842041016, y

In [ ]:
# Calculate actual distance travelled using points on trajectory (for chosen player)

In [1]:
import json
import numpy as np
import pandas as pd
import parse_data.prepare_data as prepare_data
import globals
import scipy
import math
import data_extraction.get_indices as get_indices
from assign_coordinates_to_walls import alcove_x, alcove_y
from parse_data import preprocess
data_folder = '/Users/benny/Desktop/MSc/Project/Git/repos/octagon_analysis/Json data'
json_filenames = ['2024-09-13_11-31-00_YansuJerrySocial.json']

df, trial_list = prepare_data.prepare_data(data_folder, json_filenames)

# main function summing over euclidian distances between points on trajectory
def path_distance(trial_index, chosen_player = None):
  
    this_trial = trial_list[trial_index]

    # isolate slice onset event, trigger event, and activating client
    slice_onset_event = this_trial[this_trial['eventDescription'] == globals.SLICE_ONSET]
    trigger_event = this_trial[this_trial['eventDescription'] == globals.SELECTED_TRIGGER_ACTIVATION]
    
    # find index of slice onset and trigger event normalised to this trial
    slice_onset_idx = slice_onset_event.index[0]
    slice_onset_idx = int(slice_onset_idx - this_trial.index[0])
    trigger_idx = trigger_event.index[0]
    trigger_idx = int(trigger_idx - this_trial.index[0])

    num_players = preprocess.num_players(this_trial)

    # define trajectory as the set of points on the path
    # create list containing labels for player coordinates from slice onset to trigger event
    coordinate_array_labels = []
    for i in range(num_players):
        coordinate_array_labels.extend((globals.PLAYER_LOC_DICT[i]['xloc'], globals.PLAYER_LOC_DICT[i]['yloc'])) 
    coordinate_arrays = {label : this_trial[label].values[slice_onset_idx:trigger_idx] for label in coordinate_array_labels}
    # maybe update above coordinate_array_labels?

    distances = []
    for i in range(num_players):
        if chosen_player is None:
           x = coordinate_arrays[globals.PLAYER_LOC_DICT[i]['xloc']]  # Player's x position
           y = coordinate_arrays[globals.PLAYER_LOC_DICT[i]['yloc']]  # Player's y position
           trajectory = np.array([x,y])
        
           distance = np.sum(np.linalg.norm(np.diff(trajectory, axis=1), axis=0))
           distances.append(distance)
        
           print(f"Trial {trial_index}, Player {i}: x = {x}, y = {y}, distance = {distance}")
            
        else:
           if i != chosen_player:
              pass
           else:
              x = coordinate_arrays[globals.PLAYER_LOC_DICT[i]['xloc']]  # Player's x position
              y = coordinate_arrays[globals.PLAYER_LOC_DICT[i]['yloc']]  # Player's y position
              trajectory = np.array([x,y])
        
              distance = np.sum(np.linalg.norm(np.diff(trajectory, axis=1), axis=0))
              distances.append(distance)
        
              print(f"Trial {trial_index}, Player {i}: x = {x}, y = {y}, distance = {distance}")
               
    return distances
    
# TEST-------------

    
for trial_index in range(len(trial_list)): 
    try:
       distance = path_distance(trial_index, chosen_player = 0)
       print("Actual distance travelled from slice onset to trigger event:", distance)
    except Exception as e:
       print(f"Error: {e}")

Alcove X Coordinates: [12.31592235091189, 15.582040815294189, 19.726838936116845, 19.726838936116845, 15.582040815294189, 12.31592235091189, 2.309494514323318, -2.3094945143233154, -12.315922350911888, -15.582040815294185, -19.72683893611684, -19.72683893611684, -15.582040815294192, -12.315922350911897, -2.309494514323324, 2.309494514323313]
Alcove Y Coordinates: [15.582040815294189, 12.31592235091189, 2.309494514323318, -2.3094945143233154, -12.31592235091189, -15.582040815294189, -19.726838936116845, -19.726838936116845, -15.582040815294192, -12.315922350911897, -2.3094945143233216, 2.3094945143233163, 12.315922350911892, 15.582040815294189, 19.72683893611684, 19.72683893611684]
Wall 1: X Coordinates: (12.31592235091189, 15.582040815294189), Y Coordinates: (15.582040815294189, 12.31592235091189)
Wall 2: X Coordinates: (19.726838936116845, 19.726838936116845), Y Coordinates: (2.309494514323318, -2.3094945143233154)
Wall 3: X Coordinates: (15.582040815294189, 12.31592235091189), Y Coor